In [1]:
import pandas as pd
import numpy as np
from numpy.random import permutation
from keras.layers import Input, Embedding, LSTM, Dense,Dot, Flatten, Add, Concatenate
from keras.models import Model
from keras.utils import np_utils
from keras import backend as K

assert Input and Embedding and LSTM and Dense and Dot and Flatten and Add
assert Model
assert np_utils and permutation

Using TensorFlow backend.


In [2]:
class DataManager:
    def __init__(self):
        self.data = {}
        self.id = {}
        
    def read_data(self, path, name, with_label=True, read_id=False):
        
        if with_label:
            f = pd.read_csv(path)
            uid = np.array(f.iloc[:]['UserID']).reshape(-1,1)
            mid = np.array(f.iloc[:]['MovieID']).reshape(-1,1)
            rat = np.array(f.iloc[:]['Rating']).reshape(-1,1)
            #self.Data_normalization(rat)
            data = permutation(np.hstack((uid, mid, rat)))
            X = data[:, 0:2]
            Y = data[:, 2].reshape(-1,1)
            self.data[name] = [X, Y]
            
        elif not read_id:
            f = pd.read_csv(path)
            uid = np.array(f.iloc[:]['UserID']).reshape(-1,1)
            mid = np.array(f.iloc[:]['MovieID']).reshape(-1,1)
            X = np.hstack((uid, mid))
            self.data[name] = [X]
            
        else:
            ff = open(path, 'r', encoding='latin-1')
            id_n=[]
            next(ff)
            for line in ff:
                id_n.append(int(line.split('::', 1)[0]))
            maxid = max(id_n)
            self.id[name] = [id_n, maxid]
            ff.close()
    
    def Data_normalization(self, arr): # normalize train rating
        self.train_rating_std = arr.std()
        self.train_rating_mean = arr.mean()
        arr = ((arr - arr.mean()) / arr.std())
            
            
    def build_model_0(self):
        u_input = Input(shape=(1,), name='user_input')
        m_input = Input(shape=(1,), name='movie_input')
            
        e1_u = Flatten()(Embedding(output_dim=128, input_dim=self.id['user'][1],  input_length=1)(u_input))
        e2_u = Flatten()(Embedding(output_dim=1,  input_dim=self.id['user'][1],  input_length=1)(u_input))
        e1_m = Flatten()(Embedding(output_dim=128, input_dim=self.id['movie'][1], input_length=1)(m_input))
        e2_m = Flatten()(Embedding(output_dim=1,  input_dim=self.id['movie'][1], input_length=1)(m_input))
            
        
        output = Add(name='output')([e2_u, e2_m, Dot(1)([e1_m, e1_u])])
        model = Model(inputs=[u_input, m_input], outputs=[output])
        #model.compile('adam', 'categorical_crossentropy')
        self.model_0 = model
    
    def build_model_TA(self, n_users=6040, n_items=3883, latent_dim=1024):
        user_input = Input(shape=[1], name='user_input')
        item_input = Input(shape=[1], name='movie_input')
        user_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(user_input)
        user_vec = Flatten()(user_vec)
        item_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(item_input)
        item_vec = Flatten()(item_vec)
        user_bias = Embedding(n_users, 1, embeddings_initializer='zeros')(user_input)
        user_bias = Flatten()(user_bias)
        item_bias = Embedding(n_items, 1, embeddings_initializer='zeros')(item_input)
        item_bias = Flatten()(item_bias)
        r_hat = Dot(axes=1)([user_vec, item_vec])
        r_hat = Add(name='output')([r_hat, user_bias, item_bias])
        model = Model([user_input, item_input], r_hat)
        self.model_1 = model
        # get embedding
        user_emb = np.array(model.layers[2].get_weights()).squeeze()
        movie_emb = np.array(model.layers[3].get_weights()).squeeze()
        print ('user embedding shape:', user_emb.shape)
        print ('movie embedding shape:', movie_emb.shape)
        np.save('save/user_emb_1.npy', user_emb)
        np.save('save/movie_emb_1.npy', movie_emb)
        
    def build_model_TA_no_bias(self, n_users=6040, n_items=3883, latent_dim=1024):
        user_input = Input(shape=[1], name='user_input')
        item_input = Input(shape=[1], name='movie_input')
        user_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(user_input)
        user_vec = Flatten()(user_vec)
        item_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(item_input)
        item_vec = Flatten()(item_vec)
        #user_bias = Embedding(n_users, 1, embeddings_initializer='zeros')(user_input)
        #user_bias = Flatten()(user_bias)
        #item_bias = Embedding(n_items, 1, embeddings_initializer='zeros')(item_input)
        #item_bias = Flatten()(item_bias)
        r_hat = Dot(axes=1, name='output')([user_vec, item_vec])
        #r_hat = Add(name='output')([r_hat, user_bias, item_bias])
        model = Model([user_input, item_input], r_hat)
        self.model_no_bias = model
        # get embedding
        user_emb = np.array(model.layers[2].get_weights()).squeeze()
        movie_emb = np.array(model.layers[3].get_weights()).squeeze()
        print ('user embedding shape:', user_emb.shape)
        print ('movie embedding shape:', movie_emb.shape)
        np.save('save/user_emb_1.npy', user_emb)
        np.save('save/movie_emb_1.npy', movie_emb)
        
    def build_model_TA_DNN(self, n_users=6040, n_items=3883, latent_dim=1024):
        user_input = Input(shape=[1], name='user_input')
        item_input = Input(shape=[1], name='movie_input')
        user_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(user_input)
        user_vec = Flatten()(user_vec)
        item_vec = Embedding(n_users, latent_dim, 
                             embeddings_initializer='random_normal')(item_input)
        item_vec = Flatten()(item_vec)
        merge_vec = Concatenate()([user_vec, item_vec])
        hidden = Dense(128, activation='relu')(merge_vec)
        hidden = Dense(64, activation='relu')(hidden)
        output = Dense(1, name='output')(hidden)
        model = Model([user_input, item_input], output)
        self.model_dnn = model
        # get embedding
        user_emb = np.array(model.layers[2].get_weights()).squeeze()
        movie_emb = np.array(model.layers[3].get_weights()).squeeze()
        print ('user embedding shape:', user_emb.shape)
        print ('movie embedding shape:', movie_emb.shape)
        np.save('save/user_emb_2.npy', user_emb)
        np.save('save/movie_emb_2.npy', movie_emb)
            
    def MSE(self, y_true, y_pred):
        return (K.mean(K.square(y_pred - y_true), axis=-1)) 
    
    def write_file(self, test, path):
        idx = np.array([[j for j in range(1, len(test)+1)]]).T
        test = np.hstack((idx, test))
        
        out = pd.DataFrame(test, columns=['TestDataID', 'Rating'])
        out['TestDataID'] = out['TestDataID'].astype(int)
        out.to_csv(path, index=False)
                                                                                           

In [4]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import sys
#from util import DataManager
import keras 
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(tf.Session(config=config))

sys.path.append('./data')

n_batch=2048

def main(_model):

    dm = DataManager()
    dm.read_data('data/train.csv', 'train', with_label=True)
    dm.read_data('data/test.csv', 'test', with_label=False)
    dm.read_data('data/users.csv', 'user', with_label=False, read_id=True)
    dm.read_data('data/movies.csv', 'movie', with_label=False, read_id=True)
    
    dm.build_model_0()
    dm.build_model_TA()
    dm.build_model_TA_DNN()
    dm.build_model_TA_no_bias()
    if _model == 0:
        model = dm.model_0
    elif _model == 1:
        model = dm.model_1
    elif _model == 2:
        model = dm.model_dnn
    else:
        model = dm.model_no_bias
    adam = keras.optimizers.Adam(clipnorm=0.0001)
    model.compile(optimizer=adam, loss={'output': 'mean_squared_error'}, metrics=['mean_squared_error'])
    
    filepath = 'model/model_1_1024.hdf5'
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    chechpoint2 = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
    
    callbacks_list = [checkpoint1, chechpoint2]
    
    model.summary()
    
    history = model.fit({'user_input':dm.data['train'][0][:,0], 'movie_input':dm.data['train'][0][:,1]},
              {'output':dm.data['train'][1][:,0]},
             epochs=60,
             batch_size=n_batch,
             shuffle=True,
             validation_split=0.05,
             callbacks=callbacks_list,
             verbose=1)
    
    dict_history = pd.DataFrame(history.history)
    dict_history.to_csv('save/history_1024.csv')
    
    #model.save('model.hdf5')
    
    test_y = model.predict({'user_input':dm.data['test'][0][:,0], 'movie_input':dm.data['test'][0][:,1]},
                          batch_size=n_batch, verbose=1)
    # data normalization
    #test_y = test_y * dm.train_rating_std + dm.train_rating_mean
    np.save('test_y_1_1024.npy', test_y)
    dm.write_file(test_y, './output_1_1024.csv')
    
    


In [5]:
main(1)

user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 1024)       6184960     user_input[0][0]                 
________________________________________________________________________________

InvalidArgumentError: indices[277,0] = 3948 is not in [0, 3883)
	 [[Node: embedding_8/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_8/embeddings/read, embedding_8/Cast)]]

Caused by op 'embedding_8/Gather', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipyke

In [23]:
main(3) # 1024, no bias

user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_41 (Embedding)         (None, 1, 1024)       6184960     user_input[0][0]                 
________________________________________________________________________________

In [17]:
main(1)# 1024, normal

user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_25 (Embedding)         (None, 1, 1024)       6184960     user_input[0][0]                 
____________________________________________________________________________________________________
embedding_26 (Embedding)         (None, 1, 1024)  

In [17]:
main(1)#8192

user embedding shape: (6040, 8192)
movie embedding shape: (6040, 8192)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_45 (Embedding)         (None, 1, 8192)       49479680    user_input[0][0]                 
____________________________________________________________________________________________________
embedding_46 (Embedding)         (None, 1, 8192)  

In [53]:
main(1)#4

user embedding shape: (6040, 4)
movie embedding shape: (6040, 4)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_153 (Embedding)        (None, 1, 4)          24160       user_input[0][0]                 
____________________________________________________________________________________________________
embedding_154 (Embedding)        (None, 1, 4)          2

854879/854879 [==============================] - 10s - loss: 0.7712 - mean_squared_error: 0.7712 - val_loss: 0.7992 - val_mean_squared_error: 0.7992
Epoch 38/60
854879/854879 [==============================] - 10s - loss: 0.7675 - mean_squared_error: 0.7675 - val_loss: 0.7965 - val_mean_squared_error: 0.7965
Epoch 39/60
854879/854879 [==============================] - 10s - loss: 0.7638 - mean_squared_error: 0.7638 - val_loss: 0.7941 - val_mean_squared_error: 0.7941
Epoch 40/60
854879/854879 [==============================] - 10s - loss: 0.7602 - mean_squared_error: 0.7602 - val_loss: 0.7918 - val_mean_squared_error: 0.7918
Epoch 41/60
854879/854879 [==============================] - 10s - loss: 0.7567 - mean_squared_error: 0.7567 - val_loss: 0.7899 - val_mean_squared_error: 0.7899
Epoch 42/60
854879/854879 [==============================] - 10s - loss: 0.7533 - mean_squared_error: 0.7533 - val_loss: 0.7872 - val_mean_squared_error: 0.7872
Epoch 43/60
854879/854879 [===================

In [50]:
main(1)#32

user embedding shape: (6040, 32)
movie embedding shape: (6040, 32)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_143 (Embedding)        (None, 1, 32)         193280      user_input[0][0]                 
____________________________________________________________________________________________________
embedding_144 (Embedding)        (None, 1, 32)        

854879/854879 [==============================] - 10s - loss: 0.6653 - mean_squared_error: 0.6653 - val_loss: 0.7446 - val_mean_squared_error: 0.7446
Epoch 15/60
854879/854879 [==============================] - 10s - loss: 0.6529 - mean_squared_error: 0.6529 - val_loss: 0.7409 - val_mean_squared_error: 0.7409
Epoch 16/60
854879/854879 [==============================] - 10s - loss: 0.6410 - mean_squared_error: 0.6410 - val_loss: 0.7401 - val_mean_squared_error: 0.7401
Epoch 17/60
854879/854879 [==============================] - 10s - loss: 0.6296 - mean_squared_error: 0.6296 - val_loss: 0.7383 - val_mean_squared_error: 0.7383
Epoch 18/60
854879/854879 [==============================] - 10s - loss: 0.6184 - mean_squared_error: 0.6184 - val_loss: 0.7382 - val_mean_squared_error: 0.7382
Epoch 19/60
854879/854879 [==============================] - 10s - loss: 0.6075 - mean_squared_error: 0.6075 - val_loss: 0.7382 - val_mean_squared_error: 0.7382
Epoch 20/60
854879/854879 [===================

In [42]:
main(1)#1024

user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
user embedding shape: (6040, 1024)
movie embedding shape: (6040, 1024)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_113 (Embedding)        (None, 1, 1024)       6184960     user_input[0][0]                 
____________________________________________________________________________________________________
embedding_114 (Embedding)        (None, 1, 1024)  

In [31]:
main(2) # dnn

user embedding shape: (6040, 6666)
movie embedding shape: (6040, 6666)
user embedding shape: (6040, 6666)
movie embedding shape: (6040, 6666)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_77 (Embedding)         (None, 1, 6666)       40262640    user_input[0][0]                 
____________________________________________________________________________________________________
embedding_78 (Embedding)         (None, 1, 6666)  

In [20]:

main(0)

user embedding shape: (6040, 6666)
movie embedding shape: (6040, 6666)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_35 (Embedding)         (None, 1, 128)        505856      movie_input[0][0]                
____________________________________________________________________________________________________
embedding_33 (Embedding)         (None, 1, 128)        773120      user_input[0][0]                 
____________________

In [15]:
main(1) # 6666

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
movie_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_21 (Embedding)         (None, 1, 6666)       40262640    user_input[0][0]                 
____________________________________________________________________________________________________
embedding_22 (Embedding)         (None, 1, 6666)       40262640    movie_input[0][0]                
___________________________________________________________________________________________

809885/809885 [==============================] - 41s - loss: 0.0771 - mean_squared_error: 0.0771 - val_loss: 0.9272 - val_mean_squared_error: 0.9272
Epoch 16/60
809885/809885 [==============================] - 41s - loss: 0.0783 - mean_squared_error: 0.0783 - val_loss: 0.9212 - val_mean_squared_error: 0.9212
Epoch 17/60
809885/809885 [==============================] - 41s - loss: 0.0744 - mean_squared_error: 0.0744 - val_loss: 0.9112 - val_mean_squared_error: 0.9112
Epoch 18/60
809885/809885 [==============================] - 41s - loss: 0.0756 - mean_squared_error: 0.0756 - val_loss: 0.9006 - val_mean_squared_error: 0.9006
Epoch 19/60
809885/809885 [==============================] - 41s - loss: 0.0722 - mean_squared_error: 0.0722 - val_loss: 0.8949 - val_mean_squared_error: 0.8949
Epoch 20/60
809885/809885 [==============================] - 41s - loss: 0.0729 - mean_squared_error: 0.0729 - val_loss: 0.8902 - val_mean_squared_error: 0.8902
Epoch 21/60
809885/809885 [===================

In [1]:
import numpy as np
#import pandas as pd
import sys
test_y=np.load('test_y_0.npy')
test_y += np.load('test_y_1_1024.npy')
test_y += np.load('test_y_1_128.npy')
test_y += np.load('test_y_1_32.npy')
test_y += np.load('test_y_1_4.npy')
test_y += np.load('test_y_1_8192.npy')
test_y += np.load('test_y_2.npy')
test_y/=7
print (test_y)
print (test_y.shape)
np.save('save/test_total.npy',test_y)

[[ 4.2158637 ]
 [ 3.95276141]
 [ 3.9234283 ]
 ..., 
 [ 1.63274825]
 [ 3.24265885]
 [ 4.03903723]]
(100336, 1)


In [8]:
dm = DataManager()
dm.write_file(np.load('save/test_total.npy'), './output.csv')

In [1]:
f = pd.read_csv('data/train.csv')
#print (f)
uid = np.array(f.iloc[:]['UserID']).reshape(-1,1)
mid = np.array(f.iloc[:]['MovieID']).reshape(-1,1)
rat = np.array(f.iloc[:]['Rating']).reshape(-1,1)

#print (uid)
#print ('==============')
#print (np.hstack((uid, mid, rat)))
#print ('==============')
#print (permutation(uid))

NameError: name 'pd' is not defined